<a href="https://colab.research.google.com/github/nasiksami/Natural-Language-Processing/blob/main/Extractive_text_summarization_using_TF_IDF_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rouge

In [ ]:
#importing all necessary libraries
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import reuters
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk.data
import math
import re
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge 
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
SUMMARY_LENGTH = 5  # number of sentences in final summary

stop_words = stopwords.words('english') #getting all the stopwords
ideal_sent_length = 20.0
stemmer = PorterStemmer()  #using PorterStemmer for stemming purposes

In [ ]:
class Summarizer():

    #taking in two articles and extracting the headline and body to place them in a list
    def __init__(self, article):
      self._articles = []
      for doc in article:
            with open(doc) as f:
                headline = f.readline() #first line of article is the headline
                url = f.readline() #second line of article is the url
                body = f.read().replace('\n', ' ') #read the remaining of the article and replace the empty lines with a whitespace
                #if headline and body is not empty, then we assign the values into 'articles' list
                if not self.valid_input(headline, body):
                    self._articles.append((None, None))
                    continue
                self._articles.append((headline, body))
 
    
    #check if headline and body has any text or not
    def valid_input(self, headline, article_text):
        return headline != '' and article_text != ''


    #function for generating the summary
    def generate_summaries(self):
        
        #Identifying the number of sentences in the list (containing two articles)
        total_num_sentences = 0
        for article in self._articles:
            total_num_sentences += len(self.split_into_sentences(article[1]))
        
        #If article is shorter than the desired summary, just return the original articles
        if total_num_sentences <= SUMMARY_LENGTH:
            return [x[1] for x in self._articles]

        self.build_TFIDF_model()  #build tf-idf score matrix from Reuters corpus

        self._scores = Counter()
        for article in self._articles:
            self.score(article)

        highest_scoring = self._scores.most_common(SUMMARY_LENGTH)

        #print("## Headlines: ")
        #for article in self._articles:
        #    print("- " + article[0])

        #print("\n## Summary: ")
        # Appends highest scoring "representative" sentences, returns as a single summary paragraph.
        #file to save the result
        
        with open("result.txt", "w") as fwrite:
          fwrite.write("## Headlines: \n")
          for article in self._articles:
            fwrite.write("- ")
            fwrite.write(article[0])
          fwrite.write("\n## Summary: \n")
          fwrite.write(' '.join([sent[0] for sent in highest_scoring])) 
        return ' '.join([sent[0] for sent in highest_scoring])

    
    
    #building tf_idf score in a matrix for each word in the Reuters corpus
    def build_TFIDF_model(self):
        token_dict = {}
        #getiing the name of each article and its content
        for article in reuters.fileids():
            token_dict[article] = reuters.raw(article) 

        # Use TF-IDF to determine frequency of each word in our article, relative to the
        # word frequency distributions in corpus of 10.8k Reuters news articles.
        self._tfidf = TfidfVectorizer(tokenizer=self.tokenize_and_stem, stop_words='english', decode_error='ignore')
        tdm = self._tfidf.fit_transform(token_dict.values())  # Term-document matrix
        

    #perform tokenization and stemming (using PorterStemmer)  
    def tokenize_and_stem(self, text):
        tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
        filtered = []

        #filter out numeric tokens, raw punctuation, etc.
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered.append(token)
        stems = [stemmer.stem(t) for t in filtered]
        return stems

 
    #Assigns each sentence in the document a score based on the sum of features values.
    #Based on 4 features: relevance to headline, length, sentence position, and TF*IDF frequency.
    def score(self, article):
        
        headline = article[0]
        sentences = self.split_into_sentences(article[1])
        frequency_scores = self.frequency_scores(article[1])

        for i, s in enumerate(sentences):
            headline_score = self.headline_score(headline, s) * 1.5
            length_score = self.length_score(self.split_into_words(s)) * 1.0
            position_score = self.position_score(float(i+1), len(sentences)) * 1.0
            frequency_score = frequency_scores[i] * 4
            score = (headline_score + frequency_score + length_score + position_score) / 4.0
            self._scores[s] = score

     
    #extracting each sentence
    def split_into_sentences(self, text):
        #identify where each sentence ends
        tok = nltk.data.load('tokenizers/punkt/english.pickle')
        sentences = tok.tokenize(self.remove_smart_quotes(text))
        #consider sentences with a length of more than 10
        sentences = [sent.replace('\n', '') for sent in sentences if len(sent) > 10]
        return sentences
    
    #identify and remove smart quotes
    def remove_smart_quotes(self, text):
        return text.replace(u"\u201c","").replace(u"\u201d", "")


    #Individual (stemmed) word weights are then calculated for each
    #word in the given article. Sentences are scored as the sum of their TF-IDF word frequencies.    
    def frequency_scores(self, article_text):

        # Add our document into the model so we can retrieve scores
        response = self._tfidf.transform([article_text])
        feature_names = self._tfidf.get_feature_names() # these are just stemmed words

        word_prob = {}  # TF-IDF individual word probabilities
        for col in response.nonzero()[1]:
            word_prob[feature_names[col]] = response[0, col]

        #Taking each sentence score based on its word probability
        sent_scores = []
        for sentence in self.split_into_sentences(article_text):
            score = 0
            sent_tokens = self.tokenize_and_stem(sentence)
            for token in (t for t in sent_tokens if t in word_prob):
                score += word_prob[token]

            # Normalize score by length of sentence, since we later factor in sentence length as a feature
            sent_scores.append(score / len(sent_tokens))

        return sent_scores

    #Gives sentence a score between 0 to 1 based on percentage of words common to the headline. 
    def headline_score(self, headline, sentence):
        title_stems = [stemmer.stem(w) for w in headline if w not in stop_words]
        sentence_stems = [stemmer.stem(w) for w in sentence if w not in stop_words]
        count = 0.0
        for word in sentence_stems:
            if word in title_stems:
                count += 1.0
        score = count / len(title_stems)
        return score
  
    #Gives sentence score between (0,1) based on how close sentence's length is to the ideal length. 
    def length_score(self, sentence):
        len_diff = math.fabs(ideal_sent_length - len(sentence))
        return len_diff / ideal_sent_length
      
      
    #Split a sentence string into an array of words
    def split_into_words(self, text):
        try:
            text = re.sub(r'[^\w ]', '', text) # remove non-words
            return [w.strip('.').lower() for w in text.split()]
        except TypeError:
            return None

    #Yields a value between (0,1), corresponding to sentence's position in the article.
    #Assuming that sentences at the very beginning and ends of the article have a higher weight. 
    #Values borrowed from https://github.com/xiaoxu193/PyTeaser  
    def position_score(self, i, size):

        relative_position = i / size
        if 0 < relative_position <= 0.1:
            return 0.17
        elif 0.1 < relative_position <= 0.2:
            return 0.23
        elif 0.2 < relative_position <= 0.3:
            return 0.14
        elif 0.3 < relative_position <= 0.4:
            return 0.08
        elif 0.4 < relative_position <= 0.5:
            return 0.05
        elif 0.5 < relative_position <= 0.6:
            return 0.04
        elif 0.6 < relative_position <= 0.7:
            return 0.06
        elif 0.7 < relative_position <= 0.8:
            return 0.04
        elif 0.8 < relative_position <= 0.9:
            return 0.04
        elif 0.9 < relative_position <= 1.0:
            return 0.15
        else:
            return 0   

In [ ]:
clmt_articles = ["climate-nyt.txt", "climate-npr.txt"] #getting two articles into the list
clmt_sum = Summarizer(clmt_articles) #Passing those two articles into the 'Summarizer' class
s = clmt_sum.generate_summaries() #Print the summary
print(s)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

 Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday. Though the foundation of the Saudi economy could be threatened by an accord intended to turn the world away from fossil fuels like oil, Saudi Arabia is a desert nation that could also experience more devastating heat as a result of global warming. The United States and China have jointly pledged to cut greenhouse gas emissions, but the United States has been pushing for aggressive terms requiring countries to verify their reduced emissions in a step that the Chinese see as intrusive. The president spoke by phone with Prime Minister Narendra Modi of India on Tuesday about an agreement under consideration here and conferred by phone this week with President Dilma Rousseff of Brazil, Mr. Kerry said. Some countries at the summit accus

In [ ]:
#http://textsummarization.net/text-summarizer

#summary length 5
r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday. The president spoke by phone with Prime Minister Narendra Modi of India on Tuesday about an agreement under consideration here and conferred by phone this week with President Dilma Rousseff of Brazil, Mr. Kerry said. India and Brazil, along with China and the United States, are among the world’s top producers of greenhouse gases that warm the planet. The role of the four nations in reaching a deal to curb those emissions is crucial to the success of the negotiations.'

#summary length 3
#r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday. The role of the four nations in reaching a deal to curb those emissions is crucial to the success of the negotiations. India and Brazil, along with China and the United States, are among the world’s top producers of greenhouse gases that warm the planet.'

#summary length 8
#r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday. The president spoke by phone with Prime Minister Narendra Modi of India on Tuesday about an agreement under consideration here and conferred by phone this week with President Dilma Rousseff of Brazil, Mr. Kerry said. India and Brazil, along with China and the United States, are among the world’s top producers of greenhouse gases that warm the planet. The role of the four nations in reaching a deal to curb those emissions is crucial to the success of the negotiations.'

print('Individual 1-gram: %f' % sentence_bleu(r, s, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(r, s, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(r, s, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(r, s, weights=(0, 0, 0, 1)))

#The weights for the BLEU-4 are 1/4 (25%) or 0.25 for each of the 1-gram, 2-gram, 3-gram and 4-gram scores
score = sentence_bleu(r, s, weights=(0.25, 0.25, 0.25, 0.25))
print('BLEU 4 Score: ', score)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Individual 1-gram: 0.034314
Individual 2-gram: 1.000000
Individual 3-gram: 1.000000
Individual 4-gram: 1.000000
BLEU 4 Score:  0.430394752998613


In [ ]:
#https://www.tools4noobs.com/summarize/
r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday.'

print('Individual 1-gram: %f' % sentence_bleu(r, s, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(r, s, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(r, s, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(r, s, weights=(0, 0, 0, 1)))

#The weights for the BLEU-4 are 1/4 (25%) or 0.25 for each of the 1-gram, 2-gram, 3-gram and 4-gram scores
score = sentence_bleu(r, s, weights=(0.25, 0.25, 0.25, 0.25))
print('BLEU 4 Score: ', score)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Individual 1-gram: 0.031863
Individual 2-gram: 1.000000
Individual 3-gram: 1.000000
Individual 4-gram: 1.000000
BLEU 4 Score:  0.4224942447227585


In [ ]:
#http://textsummarization.net/text-summarizer

#summary length 5
r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday. The president spoke by phone with Prime Minister Narendra Modi of India on Tuesday about an agreement under consideration here and conferred by phone this week with President Dilma Rousseff of Brazil, Mr. Kerry said. India and Brazil, along with China and the United States, are among the world’s top producers of greenhouse gases that warm the planet. The role of the four nations in reaching a deal to curb those emissions is crucial to the success of the negotiations.'

#summary length 3
#r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday. The role of the four nations in reaching a deal to curb those emissions is crucial to the success of the negotiations. India and Brazil, along with China and the United States, are among the world’s top producers of greenhouse gases that warm the planet.'

#summary length 8
#r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday. The president spoke by phone with Prime Minister Narendra Modi of India on Tuesday about an agreement under consideration here and conferred by phone this week with President Dilma Rousseff of Brazil, Mr. Kerry said. India and Brazil, along with China and the United States, are among the world’s top producers of greenhouse gases that warm the planet. The role of the four nations in reaching a deal to curb those emissions is crucial to the success of the negotiations.'

rouge = Rouge()
scores = rouge.get_scores(s, r)

print(scores)


[{'rouge-1': {'f': 0.5665236004494466, 'p': 0.45517241379310347, 'r': 0.75}, 'rouge-2': {'f': 0.45016076701068025, 'p': 0.36082474226804123, 'r': 0.5982905982905983}, 'rouge-l': {'f': 0.5013234240282473, 'p': 0.4482758620689655, 'r': 0.7386363636363636}}]


In [ ]:
#https://www.tools4noobs.com/summarize/
r = 'LE BOURGET, France — Despite a domestic agenda consumed by his response to last week’s terrorist attack in California, President Obama is closely tracking the United Nations climate change negotiations outside Paris, Secretary of State John Kerry said in an interview on Tuesday.'

rouge = Rouge()
scores = rouge.get_scores(s, r)
print(scores)

[{'rouge-1': {'f': 0.4171122959821556, 'p': 0.2689655172413793, 'r': 0.9285714285714286}, 'rouge-2': {'f': 0.3220338953792014, 'p': 0.1958762886597938, 'r': 0.9047619047619048}, 'rouge-l': {'f': 0.28461501265052624, 'p': 0.2689655172413793, 'r': 0.9285714285714286}}]


In [ ]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords
from sys import argv
    

def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary


'''
We already have a sentence tokenizer, so we just need 
to run the sent_tokenize() method to create the array of sentences.
'''

if __name__ == '__main__':

    data=open(argv[1],"r")
    sentences = sent_tokenize(f.read())


    print("\n\n\n")
    # 1 Sentence Tokenize
    
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)


    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)



    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)
    print(summary)